# Sentence Pairs
To do - 
Predict Contradiction / Non - Contradiction of pair of statements

In [2]:
# Importing the libraries
import numpy as np
import pandas as pd

# Visualisation libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from pandas.plotting import scatter_matrix

# Increase the default plot size and set the color scheme
plt.rcParams['figure.figsize'] = 8, 5
plt.style.use("fivethirtyeight")

# Disable warnings 
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Loading the dataset
df = pd.read_csv('./data/quora_pairs/dataset.txt', header = None, sep = "\t").drop(3, axis = 1)
# df = pd.read_fwf('./data/quora_pairs/dataset.txt', sep = "\t")

df.head()

,0,1,2,4
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,NEUTRAL


In [4]:
# Renaming the columns
df.columns = ['Id', 'statement_1', 'statement_2', 'label']

In [5]:
# Checking the value counts
df.label.value_counts()

NEUTRAL          2536
ENTAILMENT       1299
CONTRADICTION     665
Name: label, dtype: int64

In [6]:
df.label[2]

'ENTAILMENT'

In [7]:
# Changing the neutral and entailment to non-contradiction
df.loc[(df.label == 'NEUTRAL') | (df.label == 'ENTAILMENT'), 'label'] = 'NON-CONTRADICTION'

In [8]:
df.head()

,Id,statement_1,statement_2,label
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,NON-CONTRADICTION
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,NON-CONTRADICTION
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,NON-CONTRADICTION
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,NON-CONTRADICTION
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,NON-CONTRADICTION


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id           4500 non-null   int64 
 1   statement_1  4500 non-null   object
 2   statement_2  4500 non-null   object
 3   label        4500 non-null   object
dtypes: int64(1), object(3)
memory usage: 140.8+ KB
